In [12]:
import configuration

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
import numpy as np

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

import dataset_dataloader

In [13]:
import random

In [14]:
random.seed(1234)

## create data loader object
## image transformation function

In [3]:
from torchvision import transforms as T
import random
from PIL import Image
import torch
import os


from configuration import *
from  dataset_dataloader import *
from cross_modal_cvae_model import *

In [10]:
image_dir='/home/ubuntu/project2/StarGAN/data/celeba/images'
attr_path='/home/ubuntu/project2/StarGAN/data/celeba/list_attr_celeba.txt'
selected_attrs=['Male' ,'Young']
target_dir_MY = '/home/ubuntu/project2/CELEBA_DATA/MY'
target_dir_FY = '/home/ubuntu/project2/CELEBA_DATA/FY'
target_dir_MO = '/home/ubuntu/project2/CELEBA_DATA/MO'
target_dir_FO = '/home/ubuntu/project2/CELEBA_DATA/FO'
mode = 'train'
batch_size = 5
num_workers=1
crop_size=178
image_size=64

transform = []
if mode == 'train':
    transform.append(T.RandomHorizontalFlip())
transform.append(T.CenterCrop(crop_size))
transform.append(T.Resize(image_size))
transform.append(T.ToTensor())
#transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
transform = T.Compose(transform)

In [11]:
dataset = CelebA(image_dir, attr_path, selected_attrs, transform, mode)
data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
data_iter = iter(data_loader)
start_iters = 0
num_iters = 10

Finished preprocessing the CelebA dataset...


## create model

In [5]:
a=torch.distributions.normal.Normal(torch.tensor([0.0]), torch.tensor([1.0]))
b=a.sample((1,8,1,1))
xx=b.squeeze(4).to(DEVICE)

In [6]:
CM_gen = CrossModal(lambda_p, lambda_s, xx)
CM_gen = CM_gen.to(DEVICE)
CM_gen_optimizer = torch.optim.Adam(CM_gen.parameters(), learning_rate)

In [7]:
def init_weights(m):
    if type(m) == nn.Conv2d or type(m) == nn.Linear or type(m) == nn.ConvTranspose2d:
        torch.nn.init.xavier_normal_(m.weight.data)
        
CM_gen.apply(init_weights)

CrossModal(
  (Utterance_Encoder): UtteranceEncoder(
    (conv1): Conv2d(1, 8, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
    (conv1_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_gated): Conv2d(1, 8, kernel_size=(3, 9), stride=(1, 1), padding=(1, 4))
    (conv1_gated_bn): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1_sigmoid): Sigmoid()
    (conv2): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
    (conv2_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_gated): Conv2d(8, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
    (conv2_gated_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2_sigmoid): Sigmoid()
    (conv3): Conv2d(16, 16, kernel_size=(4, 8), stride=(2, 2), padding=(1, 3))
    (conv3_bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=

In [8]:
def label2onehot(labels, dim):
       """Convert label indices to one-hot vectors."""
       batch_size = labels.size(0)
       out = torch.zeros(batch_size, dim)
       out[np.arange(batch_size), labels.long()] = 1
       return out.to(DEVICE)

## train model

In [ ]:
num_epochs=6

for epoch in range(0, num_epochs):
    epoch += 1
    
    if (epoch == 7):
        learning_rate = learning_rate_ 
        for param_group in CM_gen_optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 15):
        learning_rate = learning_rate__
        for param_group in CM_gen_optimizer.param_groups:
            param_group['lr'] = learning_rate
    if (epoch == 21):
        learning_rate = learning_rate___
        for param_group in CM_gen_optimizer.param_groups:
            param_group['lr'] = learning_rate
    
    start_time = time.time()

    print('Epoch: %d' % epoch)
    
    avg_loss = 0
    
    data_len = len(data_iter)
    
    data_iter = iter(data_loader)

    for (x_face, label_face) in data_iter:
       
        x_voice, label_voice = data_load_new(batchsize = 20,s = label_face)
        
        x_face = x_face.to(DEVICE)
        label_face = label_face.to(DEVICE)
        x_voice = x_voice.to(DEVICE)
        label_voice = label_voice.to(DEVICE)
        
        label_real = label2onehot(labels=label_voice, dim=4)
        
        
        CM_gen_optimizer.zero_grad()
        
        loss = CM_gen.calculate_loss(x_voice, x_face, label_real)
        
        loss.backward()
        
        CM_gen_optimizer.step()
        
        avg_loss = avg_loss + loss
        
        
    CAVE_LOSS = avg_loss/data_len
    print("avg_loss", CAVE_LOSS)
      

    if epoch%5 == 0:
        torch.save({
                    'epoch': epoch,
                    'model_state_dict':CM_gen.state_dict(),
                    'optimizer_state_dict': CM_gen_optimizer.state_dict(),
                    'loss': CAVE_LOSS,
                    }, "model_params_ACVAE" + str(epoch) + ".tar")